In [31]:
import sqlite3
import os
import json

# Ruta relativa desde tu proyecto
db_path = "data/database/calidad_aire.db"
proyecto_root = "/home/jovyan/Documents/PROYECTO_REAL"  # Ajusta esta ruta si es necesario
db_abs_path = os.path.join(proyecto_root, db_path)

print(f"Ruta de la base de datos: {db_abs_path}")
print("-" * 50)

if not os.path.exists(db_abs_path):
    print("ERROR: La base de datos no existe!")
    print("Creando directorio si no existe...")
    os.makedirs(os.path.dirname(db_abs_path), exist_ok=True)
    
    # Crear una nueva base de datos
    conn = sqlite3.connect(db_abs_path)
    cursor = conn.cursor()
    
    # Crear tabla sensor_responses según la estructura que mostraste
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS sensor_responses (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            request_id INTEGER,
            calidad_aire_pred TEXT,
            co2_nivel TEXT,
            temperature REAL,
            humedad REAL,
            presion REAL,
            importancia_variables TEXT,
            prediccion_detalle TEXT,
            created_at TEXT
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS sensor_requests (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            device_id TEXT,
            request_data TEXT,
            processed_at TEXT,
            archived INTEGER DEFAULT 0
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS archivos_procesados (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre_archivo TEXT UNIQUE,
            fecha_procesado TEXT,
            procesado INTEGER DEFAULT 1,
            request_id INTEGER
        )
    ''')
    
    conn.commit()
    print("✓ Base de datos creada con tablas básicas")
else:
    print("✓ Base de datos encontrada")
    conn = sqlite3.connect(db_abs_path)
    cursor = conn.cursor()

print("\n" + "="*50)
print("ESTRUCTURA DE TABLAS")
print("="*50)

# Verificar estructura de todas las tablas
tablas = ['sensor_requests', 'sensor_responses', 'archivos_procesados', 'alertas_sistema']
for tabla in tablas:
    print(f"\n--- Tabla: {tabla} ---")
    try:
        cursor.execute(f"PRAGMA table_info({tabla})")
        columnas = cursor.fetchall()
        if columnas:
            print(f"Columnas ({len(columnas)}):")
            for col in columnas:
                pk = "✓" if col[5] == 1 else ""
                notnull = "NOT NULL" if col[3] == 1 else "NULL"
                dflt = f"DEFAULT {col[4]}" if col[4] else ""
                print(f"  {col[0]}. {col[1]:25} ({col[2]:10}) {pk} {notnull:8} {dflt}")
        else:
            print(f"  La tabla '{tabla}' no existe")
    except Exception as e:
        print(f"  Error: {e}")

print("\n" + "="*50)
print("DATOS EXISTENTES")
print("="*50)

# Verificar si hay datos en las tablas
for tabla in tablas:
    try:
        cursor.execute(f"SELECT COUNT(*) FROM {tabla}")
        count = cursor.fetchone()[0]
        print(f"{tabla:25}: {count:4} registros")
    except Exception as e:
        print(f"{tabla:25}: Error - {e}")

print("\n" + "="*50)
print("PRUEBA DE INSERCIÓN EN sensor_responses")
print("="*50)

# Primero, verificar si hay un request_id válido
cursor.execute("SELECT id FROM sensor_requests LIMIT 1")
request_id_existente = cursor.fetchone()

if request_id_existente:
    request_id = request_id_existente[0]
    print(f"Usando request_id existente: {request_id}")
else:
    # Crear un request de prueba
    cursor.execute('''
        INSERT INTO sensor_requests 
        (timestamp, device_id, request_data, processed_at, archived)
        VALUES (?, ?, ?, ?, ?)
    ''', (
        '2026-01-08T10:00:00',
        'device_test',
        '{"test": "data"}',
        '2026-01-08T10:00:00',
        0
    ))
    request_id = cursor.lastrowid
    conn.commit()
    print(f"Request de prueba creado con ID: {request_id}")

# Probar una inserción simple en sensor_responses
try:
    print(f"\nIntentando insertar en sensor_responses con request_id: {request_id}")
    
    test_data = (
        request_id,
        'Excelente',
        'Muy bueno',
        25.5,      # temperature
        60.0,      # humedad
        1013.25,   # presion
        json.dumps({'co2': 0.5, 'temperatura': 0.3}),
        json.dumps({'co2_ppm': 450.0, 'test': 'valor'}),
        '2026-01-08T10:31:18'
    )
    
    print(f"Datos a insertar: {test_data}")
    
    cursor.execute('''
        INSERT INTO sensor_responses 
        (request_id, calidad_aire_pred, co2_nivel, temperature, humedad, presion,
         importancia_variables, prediccion_detalle, created_at)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', test_data)
    
    conn.commit()
    print("✓ Inserción de prueba exitosa")
    print(f"  ID insertado: {cursor.lastrowid}")
    
    # Verificar la inserción
    cursor.execute("SELECT * FROM sensor_responses WHERE id = ?", (cursor.lastrowid,))
    fila = cursor.fetchone()
    
    print("\nFila insertada:")
    if fila:
        nombres_columnas = [desc[0] for desc in cursor.description]
        for i, (nombre, valor) in enumerate(zip(nombres_columnas, fila)):
            print(f"  {nombre:25}: {valor}")
    
    # Mostrar las últimas 5 inserciones
    print("\nÚltimas 5 inserciones en sensor_responses:")
    cursor.execute("SELECT id, request_id, calidad_aire_pred, co2_nivel, created_at FROM sensor_responses ORDER BY id DESC LIMIT 5")
    for fila in cursor.fetchall():
        print(f"  ID: {fila[0]}, Request: {fila[1]}, Calidad: {fila[2]}, CO2 Nivel: {fila[3]}, Fecha: {fila[4]}")
    
except Exception as e:
    print(f"\n✗ Error en inserción de prueba: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*50)
print("VERIFICACIÓN DE CLAVES FORÁNEAS")
print("="*50)

# Verificar constraints de clave foránea
try:
    cursor.execute("PRAGMA foreign_key_list(sensor_responses)")
    fks = cursor.fetchall()
    if fks:
        print("Claves foráneas en sensor_responses:")
        for fk in fks:
            print(f"  De {fk[3]} a {fk[2]}.{fk[4]} (on delete: {fk[5]}, on update: {fk[6]})")
    else:
        print("No hay claves foráneas definidas en sensor_responses")
except Exception as e:
    print(f"Error verificando claves foráneas: {e}")

print("\n" + "="*50)
print("DIAGNÓSTICO DE ERROR POSIBLE")
print("="*50)

# Verificar si hay conflictos de nombres de columnas
print("Verificando nombres de columnas sensibles...")
cursor.execute("PRAGMA table_info(sensor_responses)")
columnas_responses = [col[1].lower() for col in cursor.fetchall()]
print(f"Columnas en sensor_responses: {columnas_responses}")

# Columnas que tu código podría estar intentando usar
columnas_problema = ['co2_ppm', 'co2_value', 'co2', 'temperatura', 'humedad', 'presion']
for col in columnas_problema:
    if col in columnas_responses:
        print(f"✓ Columna '{col}' existe")
    else:
        print(f"✗ Columna '{col}' NO existe (esto podría causar error)")

conn.close()
print("\n" + "="*50)
print("DIAGNÓSTICO COMPLETADO")
print("="*50)

Ruta de la base de datos: /home/jovyan/Documents/PROYECTO_REAL\data/database/calidad_aire.db
--------------------------------------------------
ERROR: La base de datos no existe!
Creando directorio si no existe...
✓ Base de datos creada con tablas básicas

ESTRUCTURA DE TABLAS

--- Tabla: sensor_requests ---
Columnas (6):
  0. id                        (INTEGER   ) ✓ NULL     
  1. timestamp                 (TEXT      )  NULL     
  2. device_id                 (TEXT      )  NULL     
  3. request_data              (TEXT      )  NULL     
  4. processed_at              (TEXT      )  NULL     
  5. archived                  (INTEGER   )  NULL     DEFAULT 0

--- Tabla: sensor_responses ---
Columnas (10):
  0. id                        (INTEGER   ) ✓ NULL     
  1. request_id                (INTEGER   )  NULL     
  2. calidad_aire_pred         (TEXT      )  NULL     
  3. co2_nivel                 (TEXT      )  NULL     
  4. temperature               (REAL      )  NULL     
  5. humedad

In [ ]:
import os

# Ruta base
base_path = "C:/Users/solorzanor/Documents/PROYECTO_REAL"

# Estructura esperada
estructura_esperada = [
    "data/raw_json",
    "data/processed", 
    "data/database",
    "data/archive",
    "scripts",
    "config",
    "logs",
    "notebooks",
    "models",
    "reports"
]

print("VERIFICANDO ESTRUCTURA DEL PROYECTO")
print("=" * 50)

# Verificar cada carpeta
todas_ok = True
for carpeta in estructura_esperada:
    ruta_completa = os.path.join(base_path, carpeta)
    if os.path.exists(ruta_completa):
        print(f"{carpeta}")
    else:
        print(f"{carpeta} - NO EXISTE")
        todas_ok = False

print("\n" + "=" * 50)
if todas_ok:
    print("Estructura completa y correcta")
else:
    print(" Faltan algunas carpetas")
    
# Mostrar estructura completa
print("\n ESTRUCTURA COMPLETA:")
for root, dirs, files in os.walk(base_path):
    level = root.replace(base_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent} {os.path.basename(root) if os.path.basename(root) else "PROYECTO_REAL"}')
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f'{subindent}{file}')

VERIFICANDO ESTRUCTURA DEL PROYECTO
data/raw_json
data/processed
data/database
data/archive
scripts
config
logs
notebooks
models
reports

Estructura completa y correcta

 ESTRUCTURA COMPLETA:
 PROYECTO_REAL
  diagnostico_alertas_detallado.py
  Main.py
  Untitled.ipynb
   .ipynb_checkpoints
    diagnostico_alertas_detallado-checkpoint.py
    Main-checkpoint.py
    Untitled-checkpoint.ipynb
   config
    config.json
     .ipynb_checkpoints
      config-checkpoint.json
   data
    Untitled.ipynb
     .ipynb_checkpoints
      generador_datos_prueba-checkpoint.py
      Untitled-checkpoint.ipynb
     alertas
      alertas_20260107.json
      alertas_20260108.json
      alertas_20260109.json
      alertas_20260111.json
     archive
      sensor_data_20250115_083000.json
      sensor_data_20250115_124500.json
      sensor_data_20250115_183000.json
      sensor_data_20251201_011510.json
      sensor_data_20251201_013030.json
      sensor_data_20251201_024523.json
      sensor_data_20251201_0300

In [5]:
# consulta_estadisticas.py
import sqlite3
import pandas as pd
import numpy as np

db_path = 'data/database/calidad_aire.db'
conn = sqlite3.connect(db_path)

# 1. Estadísticas básicas de CO2
query_co2 = '''
SELECT 
    AVG(temperature) as temp_promedio,
    MIN(temperature) as temp_min,
    MAX(temperature) as temp_max,
    AVG(humedad) as humedad_promedio,
    MIN(humedad) as humedad_min,
    MAX(humedad) as humedad_max,
    COUNT(*) as total_muestras
FROM sensor_responses
'''
df_co2 = pd.read_sql_query(query_co2, conn)
print("ESTADÍSTICAS BÁSICAS:")
print(df_co2.to_string())

# 2. Distribución por categoría de calidad
query_calidad = '''
SELECT 
    calidad_aire_pred,
    COUNT(*) as cantidad,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM sensor_responses), 1) as porcentaje
FROM sensor_responses
GROUP BY calidad_aire_pred
ORDER BY 
    CASE calidad_aire_pred
        WHEN 'Excelente' THEN 1
        WHEN 'Buena' THEN 2
        WHEN 'Moderada' THEN 3
        WHEN 'Deficiente' THEN 4
        WHEN 'Muy deficiente' THEN 5
        WHEN 'Peligrosa' THEN 6
        ELSE 7
    END
'''
df_calidad = pd.read_sql_query(query_calidad, conn)
print("\nDISTRIBUCIÓN POR CALIDAD:")
print(df_calidad.to_string())

# 3. Por ubicación
query_ubicacion = '''
SELECT 
    json_extract(prediccion_detalle, '$.ubicacion') as ubicacion,
    COUNT(*) as muestras,
    AVG(json_extract(prediccion_detalle, '$.co2_ppm')) as co2_promedio,
    MAX(json_extract(prediccion_detalle, '$.co2_ppm')) as co2_max,
    MIN(json_extract(prediccion_detalle, '$.co2_ppm')) as co2_min
FROM sensor_responses
WHERE prediccion_detalle IS NOT NULL
GROUP BY ubicacion
ORDER BY co2_promedio DESC
'''
df_ubicacion = pd.read_sql_query(query_ubicacion, conn)
print("\nPOR UBICACIÓN:")
print(df_ubicacion.to_string())

# 4. Alertas
query_alertas = '''
SELECT 
    nivel,
    COUNT(*) as cantidad,
    SUM(CASE WHEN procesada = 1 THEN 1 ELSE 0 END) as procesadas,
    SUM(CASE WHEN procesada = 0 THEN 1 ELSE 0 END) as pendientes
FROM alertas_sistema
GROUP BY nivel
'''
df_alertas = pd.read_sql_query(query_alertas, conn)
print("\nALERTAS:")
print(df_alertas.to_string())

conn.close()

ESTADÍSTICAS BÁSICAS:
   temp_promedio  temp_min  temp_max  humedad_promedio  humedad_min  humedad_max  total_muestras
0      27.785379     19.08     35.94         79.495623        53.23         95.0             409

DISTRIBUCIÓN POR CALIDAD:
  calidad_aire_pred  cantidad  porcentaje
0         Excelente        17         4.2
1             Buena        55        13.4
2          Moderada        97        23.7
3        Deficiente       101        24.7
4    Muy deficiente        67        16.4
5         Peligrosa        72        17.6

POR UBICACIÓN:
                           ubicacion  muestras  co2_promedio  co2_max  co2_min
0            Laboratorio de Ciencias        52   1182.393654  1772.22   519.25
1                Cafetería Principal        48   1151.842500  2941.52   532.76
2    Edificio de Ingeniería - Piso 3        56   1131.874643  2519.77   498.80
3                 Biblioteca Central        48   1017.517500  2265.24   520.77
4                  Auditorio Central        44    84